In [1]:
!pip install -q tfds-nightly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 52.4 MB/s eta 0:00:00


In [2]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [3]:
ds_split, info = tfds.load("penguins/processed", split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)
ds_train = ds_split[1]
ds_test = ds_split[0]


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/penguins/processed/incomplete.85MC0P_1.0.0/penguins-train.tfrecord*...:   …

Dataset penguins downloaded and prepared to /root/tensorflow_datasets/penguins/processed/1.0.0. Subsequent calls will reuse this data.


In [4]:
df_train = tfds.as_dataframe(ds_train.take(5), info)
print(df_train)


                                         features  species
0  [0.11636364, 0.6904762, 0.2542373, 0.30555555]        0
1   [0.6290909, 0.32142857, 0.7457627, 0.6180556]        2
2  [0.13090909, 0.4642857, 0.2881356, 0.18055555]        0
3  [0.14909092, 0.35714287, 0.2542373, 0.2361111]        0
4          [0.3927273, 0.0, 0.7288136, 0.6388889]        2


In [5]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [7]:
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(model, x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_object(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


In [8]:
epochs = 5
for epoch in range(epochs):
    for features, labels in ds_train.batch(32):
        loss = train_step(model, features, labels)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.numpy()}")


Epoch 1/5, Loss: 1.2467976808547974
Epoch 2/5, Loss: 1.2363505363464355
Epoch 3/5, Loss: 1.225938320159912
Epoch 4/5, Loss: 1.2156882286071777
Epoch 5/5, Loss: 1.2056447267532349


In [9]:
test_loss = 0
for features, labels in ds_test.batch(32):
    predictions = model(features)
    test_loss += loss_object(labels, predictions).numpy()
print(f"Test Loss: {test_loss}")


Test Loss: 9.931214094161987


In [10]:
predictions = model(features)
predictions_softmax = tf.nn.softmax(predictions)
